# HyperDrive Best Run
In this notebook, we recover into Python the HyperDrive run that was created in the previous notebook, use it to find the best child run discovered by the HyperDrive search, use that best run's parameters to train a model, and finally evaluate that model on the test data.

The steps in this notebook are
- [import libraries](#import),
- [read in the Azure ML workspace](#workspace),
- [recover the HyperDrive run](#recover),
- [use the best hyperparameters found to train a model](#results), and
- [evaluate the result using the test data](#evaluate)

## Imports  <a id='import'></a>

In [ ]:
import os
import shutil
import json
import time
import pandas as pd
from azureml.core import Workspace, Experiment, Run
from azureml.train.hyperdrive import HyperDriveRun
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
import azureml.core
from msrest.exceptions import HttpOperationError
from get_auth import get_auth
print('azureml.core.VERSION={}'.format(azureml.core.VERSION))

## Read in the Azure ML workspace  <a id='workspace'></a>
Read in the the workspace created in a previous notebook.

In [ ]:
auth = get_auth()
ws = Workspace.from_config(auth=auth)
ws_details = ws.get_details()
print('Name:\t\t{}\nLocation:\t{}'
      .format(ws_details['name'],
              ws_details['location']))

## Recover the run  <a id='recover'></a>
Get an experiment that ran the search.

In [ ]:
exp = Experiment(workspace=ws, name='hypetuning')

Get the ID of the HyperDrive run created in the last notebook. That ID was printed with the run when it was submitted in the previous notebook, and we also saved it in a file. You can also find that ID in Azure Portal on your experiment's page. To see it, you may need to add a `RunId` column to the experiment's table of runs.

In [ ]:
run_id_path = "run_id.txt"
with open(run_id_path, "r") as fp:
    run_id = fp.read()
run_id

Use the ID of the HyperDrive run to get a handle to it.

In [ ]:
run = HyperDriveRun(exp, run_id)
run

## Use the best hyperparameters to train a model <a id='results'></a>
We can automatically select the best run.

In [ ]:
best_run = run.get_best_run_by_primary_metric()
best_run

Here is the best run's hyperparameter set.

In [ ]:
parameter_values = best_run.get_details()['runDefinition']['arguments']
best_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))
pd.Series(best_parameters, name='Value').to_frame()

We can use these parameters to train and save the best model. We will train with a boosted number of estimators.

In [ ]:
ds = ws.get_default_datastore()
model_parameters = best_parameters.copy()
model_parameters['--data-folder'] = ds.as_mount()
model_parameters['--estimators'] = 8 * int(best_parameters['--estimators'])
model_parameters['--save'] = 'model'
pd.Series(model_parameters, name='Value').to_frame()

Get the compute target.

In [ ]:
cluster_name = 'hypetuning'
compute_target = ws.compute_targets[cluster_name]

Train and save the best model.

In [ ]:
model_est = Estimator(source_directory=os.path.join('.', 'scripts'),
                      entry_script='TrainClassifier.py',
                      script_params=model_parameters,
                      compute_target=compute_target,
                      conda_packages=['pandas==0.23.4',
                                      'scikit-learn==0.20.0'],
                      pip_packages=['lightgbm==2.1.2'])
model_run = exp.submit(model_est)
model_run

Wait for the model to be created and saved.

In [ ]:
%%time

model_run_status = model_run.wait_for_completion()
print(model_run_status['status'])
if model_run_status['status'] not in ['Completed', 'Finalizing']:
    raise Exception('The run did not successfully complete.')

Register the best model.

In [ ]:
model = model_run.register_model(model_name='FAQ_ranker', model_path=os.path.join('outputs', 'model.pkl'))
print(model.name, model.version, sep = '\t')

## Evaluate the model using the test data <a id='evaluate'></a>
Download the model.

In [ ]:
model_run.download_file(os.path.join('outputs', 'model.pkl'),
                        os.path.join('outputs', 'FAQ_ranker.pkl'))

Look at the model's performance on the held-aside test data. This can take a couple of minutes.

In [ ]:
%run -t scripts/TestClassifier.py --model FAQ_ranker

The [next notebook](06_Tear_Down.ipynb) shows how to delete the components of this tutorial.